In [0]:
# Imports
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic
from pyspark.sql.functions import when
from pyspark.sql.functions import col
import random as r
from random import randint

In [0]:
# Get error message functions
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
%fs

ls /mnt/julianngroglio/school-finance

path,name,size,modificationTime
dbfs:/mnt/julianngroglio/school-finance/census-clean/,census-clean/,0,1660059215000
dbfs:/mnt/julianngroglio/school-finance/census-data,census-data,7255007,1659469854000
dbfs:/mnt/julianngroglio/school-finance/demo-clean/,demo-clean/,0,1660059217000
dbfs:/mnt/julianngroglio/school-finance/demographic-factors,demographic-factors,93703,1660055843000
dbfs:/mnt/julianngroglio/school-finance/grad-clean/,grad-clean/,0,1660059216000
dbfs:/mnt/julianngroglio/school-finance/grad-rate,grad-rate,166631,1659993188000
dbfs:/mnt/julianngroglio/school-finance/name-changes,name-changes,20054,1660155004000


In [0]:
censusdf = spark.read.option("header", "true").csv("/mnt/julianngroglio/school-finance/census-clean")

In [0]:
confluentClusterName = "stage3talent"
confluentBootstrapServers = 
confluentTopicName = "school-finances"
schemaRegistryUrl = 
confluentApiKey = 
confluentSecret = 
confluentRegistryApiKey = 
confluentRegistrySecret = 

In [0]:
#Kakfa Class Setup.
p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(1),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
topic_list = []

topic_list.append(NewTopic(confluentTopicName, 1, 3))
admin_client.create_topics(topic_list)
futures = admin_client.create_topics(topic_list)


try:
    record_metadata = []
    for k, future in futures.items():
        # f = i.get(timeout=10)
        print(f"type(k): {type(k)}")
        print(f"type(v): {type(future)}")
        print(future.result())

except KafkaError as error:
    # Decide what to do if produce request failed...
    print(traceback.format_exc())
    result = 'Fail'
except Exception as e:
    print(e)
    pass
finally:
    print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
KafkaError{code=TOPIC_ALREADY_EXISTS,val=36,str="Topic 'school-finances' already exists."}
finally

In [0]:
def sendrow(x):
    p.produce(confluentTopicName,json.dumps(x))
    p.flush()
   
censuspd = censusdf.toPandas()
censuspd.apply(lambda x: sendrow(x.to_dict()), axis=1)

Out[8]: 0 None
1 None
2 None
3 None
4 None
 ... 
673 None
674 None
675 None
676 None
677 None
Length: 678, dtype: object